In [61]:
sample_text = 'CWE-200 static int mincore_pte_range ( pmd_t * pmd , unsigned long addr , unsigned long end , struct mm_walk * walk ) { spinlock_t * ptl ; struct vm_area_struct * vma = walk -> vma ; pte_t * ptep ; unsigned char * vec = walk -> private ; int nr = ( end - addr ) >> PAGE_SHIFT ; ptl = pmd_trans_huge_lock ( pmd , vma ) ; if ( ptl ) { memset ( vec , 1 , nr ) ; spin_unlock ( ptl ) ; goto out ; } if ( pmd_trans_unstable ( pmd ) ) { <S2SV_StartBug> __mincore_unmapped_range ( addr , end , vma , vec ) ; <S2SV_EndBug> goto out ; } ptep = pte_offset_map_lock ( walk -> mm , pmd , addr , & ptl ) ; for ( ; addr != end ; ptep ++ , addr += PAGE_SIZE ) { pte_t pte = * ptep ; if ( pte_none ( pte ) ) <S2SV_StartBug> __mincore_unmapped_range ( addr , addr + PAGE_SIZE , <S2SV_EndBug> vma , vec ) ; else if ( pte_present ( pte ) ) * vec = 1 ; else { swp_entry_t entry = pte_to_swp_entry ( pte ) ; <S2SV_StartBug> if ( non_swap_entry ( entry ) ) { <S2SV_EndBug> * vec = 1 ; } else { # ifdef CONFIG_SWAP * vec = mincore_page ( swap_address_space ( entry ) , swp_offset ( entry ) ) ; # else WARN_ON ( 1 ) ; * vec = 1 ; # endif } } vec ++ ; } pte_unmap_unlock ( ptep - 1 , ptl ) ; out : walk -> private += nr ; cond_resched ( ) ; return 0 ; }'

In [62]:
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [64]:
def create_dummy_data_file(tokenizer):
    for file_type in ['train','valid']:
        for file_cls in ['src', 'tgt']:
            with open(f'new_random_fine_tune_{file_type}.{file_cls}.txt'.format(file_type, file_cls), 'w+') as sf:
                with open(f'/home/lgm/VRepair2.0/vul_data/random_fine_tune_{file_type}.{file_cls}.txt'.format(file_type, file_cls)) as file:
                    data = file.readlines(20)
                    for line in data:
                        line = line.strip()
                        tokens = [tokenizer.cls_token]+tokenizer.tokenize(line)+[tokenizer.eos_token]
                        sf.writelines(' '.join(tokens))

In [148]:
create_dummy_data_file(tokenizer)

In [181]:
# %%time
PATH = '/home/lgm/VRepair2.0/param_sweep_tgt/00_parameter_sweep/'
def read_opennmt_vocab(vocab_file=None):
    if vocab_file is None:
        vul_vocab = []
        with open(PATH+'data.vocab.src') as file:
            data = file.readlines()
            for line in data:
                line = line.strip().split('\t')
                vul_vocab.append(line[0])
        with open(PATH+'data.vocab.tgt') as file:
            data = file.readlines()
            for line in data:
                line = line.strip().split('\t')
                vul_vocab.append(line[0])
        vul_vocab = list(set(vul_vocab))
        with open(PATH+'vocab.txt', 'w+') as file:
            file.write('\n'.join(vul_vocab))
        return vul_vocab

    elif vocab_file and type(vocab_file) is str:
        with open(vocab_file, 'r') as file:
            data = file.read().splitlines()
            return data

In [154]:
temp = read_opennmt_vocab()
temp

['CWE-119\t99999999\n']


['CWE-119', ')', ';']

In [142]:
%%time
import pandas as pd
import numpy as np

def create_embedding_file_v1(tokenizer, model):
    vul_vocab = read_opennmt_vocab()
    vocab_size = 10
    print("Total vocab size", len(vul_vocab))
    # vocab_dict = {}
    vocab_df = pd.DataFrame()
    vocab_df['tok'] = vul_vocab[:vocab_size]
    print(vocab_df.iloc[:vocab_size])
    with open('codebert_embeddings.txt', 'w+') as ebf:
        vul_embs = torch.Tensor(size=(vocab_size, 768))
        for idx, vocab in enumerate(vul_vocab[:vocab_size]):
            code_tokens=tokenizer.tokenize(vocab)
            tokens_ids=tokenizer.convert_tokens_to_ids(code_tokens)
            context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]
            context_embeddings = torch.mean(context_embeddings, dim=1).ravel()
            vul_embs[idx,:] = context_embeddings
        vul_embs = vul_embs.detach().numpy()
        vul_df = pd.DataFrame(vul_embs)
        df = pd.concat([vocab_df, vul_df], axis=1)
        df.to_csv(ebf, sep=' ', header=False, index=False)

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 21.9 µs


In [143]:
%%time
create_embedding_file_v1(tokenizer, model)

                                 tok
0                            CWE-125
1                                  =
2                                  {
3                            CWE-200
4                            CWE-264
5                             return
6                                 ||
7                            CWE-416
8                            CWE-119
9                      <S2SV_EndBug>
10                           CWE-415
11                                 !
12                              NULL
13                            CWE-20
14                         SecHandle
15                            size_t
16                           CWE-787
17                   <S2SV_StartBug>
18                                 *
19                           dwLower
20                                 }
21                           CWE-190
22                            handle
23                  SecIsValidHandle
24                                 ;
25                           pointer
2

In [189]:
import pandas as pd

data = pd.read_csv('codebert_embeddings.txt', sep=" ", header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,fileversion,-0.227103,0.188753,0.164321,0.196685,0.382543,-0.225033,0.179566,-0.156711,-0.002172,...,0.244525,-0.098539,-0.463918,0.486172,-0.039446,0.203790,0.455514,-0.310756,-0.485383,0.328354
1,"""use<S2SV_blank>%""",-0.253789,0.291998,0.246354,0.141815,0.468213,-0.344239,-0.038409,0.029141,-0.041933,...,0.362705,-0.039795,-0.446444,0.794468,0.028624,0.330525,0.517151,-0.590131,-0.552155,0.271466
2,MaxParameterCount,-0.299175,0.191053,0.043153,0.022680,0.630337,-0.105375,0.150037,-0.240559,0.159232,...,0.518634,0.042508,-0.666326,0.690213,-0.181200,0.223636,0.403031,-0.213640,-0.535679,0.212408
3,mdm,-0.116914,0.415617,0.089554,0.290109,0.239812,-0.633349,0.055567,0.057130,0.351040,...,0.226345,-0.174074,-0.465231,0.629849,-0.340522,0.169145,0.593982,-0.306365,-0.311245,0.276609
4,gdqp,-0.226072,0.331530,0.265502,-0.208511,0.214795,-0.321528,-0.011851,-0.009331,-0.027558,...,0.342614,-0.045061,-0.304829,0.629235,-0.292911,0.210276,0.875552,-0.268670,-0.421502,0.336709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37494,iwgifrcontext,-0.125077,0.180032,0.081595,0.290549,0.062682,-0.242019,0.166154,-0.126760,0.065995,...,0.268557,0.198563,-0.434028,0.725539,-0.225162,0.290244,0.740436,0.176292,-0.603382,0.540637
37495,EX_NOHIDE,-0.221225,0.369828,0.302403,-0.283496,0.312562,-0.248719,0.152720,0.271498,0.198392,...,0.299828,-0.025016,-0.506208,0.438071,-0.160103,0.261827,0.271013,-0.328290,-0.259278,0.386754
37496,"""bStopBits:<S2SV_blank><S2SV_blank><S2SV_blank...",-0.210213,0.182896,0.132826,0.281152,0.213210,-0.179228,-0.024944,0.003556,-0.131707,...,0.400412,0.168765,-0.348112,0.923492,-0.270259,0.363678,0.408561,-0.544400,-0.550939,0.332611
37497,"""xfs_attr_shortform_list""",-0.295012,-0.009762,0.091771,0.268375,-0.195982,0.040934,0.079474,0.020587,0.053788,...,-0.057133,-0.003092,-0.321679,0.418479,-0.210504,0.258104,0.418320,-0.575344,-0.511995,0.316794


In [191]:
vocab = read_opennmt_vocab(PATH+'vocab.txt')
# print(vocab)
vocab_df = pd.DataFrame(vocab, columns=['tok'])

In [201]:
vocab_df[vocab_df['tok'] == 'NULL']

,tok
1533,NULL


In [210]:
data.loc[1533, 0] = 'NULL'

In [216]:
data.to_csv('adjusted_codebert_emb.txt', sep=' ', header=False, index=False)

In [213]:
data.loc[:,0].values.tolist() == vocab_df['tok'].values.tolist()

True